**Colab**: This assignment is created using colab.

In [8]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,240 kB in 7s (454 kB/s)
Reading package lists... Done


In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [13]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote='\"',escape='\"')
home_sales.createOrReplaceTempView('home_sales')

home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [19]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price = spark.sql('''
    SELECT year(date) as sale_year,
    AVG(price) as average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY sale_year
    ORDER BY sale_year
    ''')
avg_price.show()



+---------+------------------+
|sale_year|     average_price|
+---------+------------------+
|     2019| 300263.6955128205|
|     2020|298353.78003169573|
|     2021|   301819.44398864|
|     2022| 296363.8845050215|
+---------+------------------+



In [21]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_3_bed_3_bath = spark.sql('''
    SELECT year(date) as sale_year,
    AVG(price) as average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY sale_year
    ORDER BY sale_year
    ''')
avg_price_3_bed_3_bath.show()



+---------+------------------+
|sale_year|     average_price|
+---------+------------------+
|     2019|287287.81696428574|
|     2020| 294204.1629104958|
|     2021| 294211.4595798854|
|     2022| 292725.6931818182|
+---------+------------------+



In [23]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_3_bed_3_bath_2_floors_2000_sqft = spark.sql('''
    SELECT year(date) as sale_year,
    AVG(price) as average_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY sale_year
    ORDER BY sale_year
    ''')

avg_price_3_bed_3_bath_2_floors_2000_sqft.show()


+---------+-----------------+
|sale_year|    average_price|
+---------+-----------------+
|     2019|289859.1354679803|
|     2020|292289.0895140665|
|     2021|296330.9612590799|
|     2022|290242.9879518072|
+---------+-----------------+



In [24]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_view = spark.sql('''
    SELECT view,
    AVG(price) as average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
    ''')

avg_price_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
|   0|403848.51461988303|
|   1| 401044.2513368984|
|   2|397389.24752475246|
|   3| 398867.5964912281|
|   4|         399631.89|
|   5| 401471.8248587571|
|   6| 395655.3789473684|
|   7| 403005.7709497207|
|   8|398592.70658682636|
|   9| 401393.3370786517|
|  10|401868.42523364484|
|  11|399548.11891891895|
|  12| 401501.3243243243|
|  13|         398917.98|
|  14|398570.02923976607|
|  15|404673.29545454547|
|  16| 399586.5311004785|
|  17|398474.49029126216|
|  18| 399332.9090909091|
|  19| 398953.1703296703|
+----+------------------+
only showing top 20 rows

--- 0.7557346820831299 seconds ---


In [25]:
# 7. Cache the the temporary table home_sales.
home_sales.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [26]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [27]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
avg_price_view_cached = spark.sql('''
    SELECT view,
    AVG(price) as average_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
    ''')

avg_price_view_cached.show()




print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     average_price|
+----+------------------+
|   0|403848.51461988303|
|   1| 401044.2513368984|
|   2|397389.24752475246|
|   3| 398867.5964912281|
|   4|         399631.89|
|   5| 401471.8248587571|
|   6| 395655.3789473684|
|   7| 403005.7709497207|
|   8|398592.70658682636|
|   9| 401393.3370786517|
|  10|401868.42523364484|
|  11|399548.11891891895|
|  12| 401501.3243243243|
|  13|         398917.98|
|  14|398570.02923976607|
|  15|404673.29545454547|
|  16| 399586.5311004785|
|  17|398474.49029126216|
|  18| 399332.9090909091|
|  19| 398953.1703296703|
+----+------------------+
only showing top 20 rows

--- 1.6205904483795166 seconds ---


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy('date_built').parquet('partitioned_home_sales.parquet')


In [29]:
# 11. Read the parquet formatted data.
partitioned_home_sales = spark.read.parquet('partitioned_home_sales.parquet')

In [30]:
# 12. Create a temporary table for the parquet data.
partitioned_home_sales.createOrReplaceTempView('partitioned_home_sales')


In [31]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

avg_price_view_parquet = spark.sql('''
    SELECT view,
    AVG(price) as average_price
    FROM partitioned_home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
    ''')

avg_price_view_parquet.show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
|   0|403848.51461988303|
|   1| 401044.2513368984|
|   2|397389.24752475246|
|   3| 398867.5964912281|
|   4|         399631.89|
|   5| 401471.8248587571|
|   6| 395655.3789473684|
|   7| 403005.7709497207|
|   8|398592.70658682636|
|   9| 401393.3370786517|
|  10|401868.42523364484|
|  11|399548.11891891895|
|  12| 401501.3243243243|
|  13|         398917.98|
|  14|398570.02923976607|
|  15|404673.29545454547|
|  16| 399586.5311004785|
|  17|398474.49029126216|
|  18| 399332.9090909091|
|  19| 398953.1703296703|
+----+------------------+
only showing top 20 rows

--- 1.1971077919006348 seconds ---


In [32]:
# 14. Uncache the home_sales temporary table.
home_sales.unpersist()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [34]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False